In [2]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor

In [3]:
import sys
import gc
import pickle

In [4]:
data = pd.read_pickle('dataframe.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6639294 entries, 4488710 to 11128003
Data columns (total 41 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   date_block_num                        int8   
 1   shop_id                               int8   
 2   item_id                               int16  
 3   item_cnt_month                        float32
 4   ID                                    float64
 5   city_code                             int8   
 6   item_category_id                      int8   
 7   type_code                             int8   
 8   subtype_code                          int8   
 9   item_cnt_month_lag_1                  float32
 10  item_cnt_month_lag_2                  float32
 11  item_cnt_month_lag_3                  float32
 12  item_cnt_month_lag_6                  float32
 13  item_cnt_month_lag_12                 float32
 14  date_avg_item_cnt_lag_1               float32
 15  date_ite

In [5]:
import xgboost as xgb


In [ ]:
X_train = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 34]['item_cnt_month']

X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

params = {'booster': 'gbtree', 'objective': 'reg:linear', 'eta': 0.1, 'gamma': 1.3, 'min_child_weight': 5.0, 'max_depth': 9.0, 'subsample': 1.0, 'colsample_bytree': 0.7000000000000001, 'num_round': 500.0, 'nthread': 2, 'silent': 0, 'seed': 2019, 'max_evals': 200}

model = xgb.sklearn.XGBRegressor(
    max_depth=int(params['max_depth']),
    learning_rate=params['eta'],
    n_estimators=int(params['num_round']),
    silent=False,
    objective=params['objective'],
    booster=params['booster'],
    nthread=int(params['nthread']),
    gamma=params['gamma'],
    min_child_weight=params['min_child_weight'],
    subsample=params['subsample'],
    colsample_bytree=params['colsample_bytree'],
    seed=params['seed']
)
model.fit(X_train, Y_train)
pred = model.predict(X_test).clip(0, 20)

[08:59:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[08:59:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[08:59:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [8]:
test=pd.read_csv(r'C:\Users\HP\Downloads\predict future sales kaggle\test.csv')

In [9]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": pred
})
submission.to_csv('submission_2.csv', index=False)